# Análise Eleitoral

### 0) Configuração dos bancos a serem utilizados

In [1]:
import pandas as pd
import tse

In [32]:
CANDIDATO = 'RICARDO NUNES'
ESTADO = 'sp' # lowercase!

In [3]:
database_1 = f'databases/{ESTADO}_2024.sql'
database_2 = 'databases/eleicoes_2024.sql'

In [4]:
analise = tse.TseAnalysis(
    database={
        'path': database_1,
        'alias': f'{ESTADO}'
    })

In [5]:
analise.attach_database(
    attach= [
        {
            'path': database_2,
            'alias': 'geral'
        }
    ])

Bancos carregados:
0) main
-path: d:\projects\tse-data\databases\sp_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
2) sp
-path: d:\projects\tse-data\databases\sp_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
3) geral
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [6]:
analise.create_index(
    table='resultados',
    columns=['NM_VOTAVEL', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO']
)
analise.create_index(
    table='eleitorado',
    columns=['NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO']
)

'Indices criados com sucesso'

---

### 1) Criação de tabela com perfil de eleitorado por seção

In [7]:
# a função generate_stats_query foi escrita para gerar uma query específica para a tabela {ESTADO}. eleitorado
# ela analisa os valores distintos de cada coluna argumento e produz uma query que soma a quantidade de aparições para cada caso
q = analise.generate_stats_query(
    alias=f'{ESTADO}',
    table='eleitorado',
    cols=[
        'DS_GRAU_ESCOLARIDADE',
        'DS_RACA_COR',
        'DS_FAIXA_ETARIA',
        'DS_GENERO'
    ]
)

In [8]:
eleitorado_secao = pd.read_sql(q, analise.conn)

In [9]:
eleitorado_secao.head(2) # este DataFrame expressa a quantidade de eleitores por seçao divido em categorias

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_superior_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_ensino_médio_completo,ds_grau_escolaridade_ensino_médio_incompleto,...,ds_faixa_etaria_85_a_89_anos,ds_faixa_etaria_65_a_69_anos,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_inválida,ds_genero_masculino,ds_genero_feminino,ds_genero_não_informado
0,71072,1015,1,1,24,231,42,21,75,27,...,27,67,8,3,12,0,0,189,241,1
1,71072,1015,1,2,14,158,32,7,82,29,...,8,29,5,2,5,0,0,149,182,2


In [10]:
q_2 = f"""
    SELECT
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        QT_ELEITOR_SECAO as eleitores_totais
    FROM
        geral.locais
    WHERE
        SG_UF = '{ESTADO.upper()}'
"""

In [11]:
eleitores_secao = pd.read_sql(q_2, analise.conn)

In [12]:
eleitores_secao.head(2) # complementa o DataFrame anterior com a quantidade total de eleitores na seção

,municipio,local,zona,secao,eleitores_totais
0,72451,1074,147,178,250
1,72451,1120,147,168,281


In [14]:
eleitorado_secao = pd.merge(
    eleitorado_secao,
    eleitores_secao,
    how='inner',
    on=[
        'municipio',
        'local',
        'zona',
        'secao'
    ]
)

In [15]:
eleitorado_secao.head(2) # resultado obtido

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_superior_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_ensino_médio_completo,ds_grau_escolaridade_ensino_médio_incompleto,...,ds_faixa_etaria_65_a_69_anos,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_inválida,ds_genero_masculino,ds_genero_feminino,ds_genero_não_informado,eleitores_totais
0,71072,1015,1,1,24,231,42,21,75,27,...,67,8,3,12,0,0,189,241,1,431
1,71072,1015,1,1,24,231,42,21,75,27,...,67,8,3,12,0,0,189,241,1,431


---

### 2) Cria tabela com apuração dos resultados por seção de determinado candidato

In [33]:
query = f"""
    SELECT
        NM_VOTAVEL as candidato,
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        NR_TURNO as turno,
        QT_VOTOS as votos_recebidos
    FROM
        {ESTADO}.resultados
    WHERE
        NM_VOTAVEL LIKE '{'%' + CANDIDATO.replace(' ', '%') + '%'}'
"""

In [34]:
resultado_secao = pd.read_sql(query, analise.conn)

In [35]:
resultado_secao.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos
0,RICARDO LUIS REIS NUNES,71072,1333,408,323,2,101
1,RICARDO LUIS REIS NUNES,71072,1090,4,79,2,152


In [36]:
resultado_perfil = pd.merge(
    resultado_secao,
    eleitorado_secao,
    how='inner',
    on=['municipio', 'local', 'zona', 'secao']
)

In [37]:
resultado_perfil.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_superior_incompleto,...,ds_faixa_etaria_65_a_69_anos,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_inválida,ds_genero_masculino,ds_genero_feminino,ds_genero_não_informado,eleitores_totais
0,RICARDO LUIS REIS NUNES,71072,1333,408,323,2,101,99,14,9,...,7,0,0,0,0,0,150,218,0,368
1,RICARDO LUIS REIS NUNES,71072,1333,408,323,2,101,99,14,9,...,7,0,0,0,0,0,150,218,0,368


In [38]:
resultado_perfil['percentual_votos'] = resultado_perfil['votos_recebidos']/resultado_perfil['eleitores_totais']

In [39]:
nome = resultado_perfil['candidato'][0]
resultado_perfil.to_csv(f'resultados_secao_absolutos/{nome.replace(' ', '_')}.csv')